<a href="https://colab.research.google.com/github/devilishAM/HomePrice-kaggle/blob/main/Model_HousePrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

#read data
X = pd.read_csv('train.csv', index_col = 'Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

#removing rows with missing target, target kept seperate
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [ ]:
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

#define the model
my_model = XGBRegressor(n_estimators= 2000, learning_rate= 0.05)

#fit the model
my_model.fit(X_train, y_train, 
             early_stopping_rounds=200, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False) 
#16026.8681640625
predictions = my_model.predict(X_valid)

print("mean absolute error is : " + str(mean_absolute_error(predictions, y_valid)) )

[18:11:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
mean absolute error is : 16073.577108304795


In [ ]:
final_pred = my_model.predict(X_test)

output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': final_pred})
output.to_csv('submission5.csv', index=False)